In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, dpi_save=300, color_map='viridis')

In [ ]:
data1 = sc.read('/home/jovyan/YS_project/YS/Data_objects/YS_object_14092021/Combined_YS_Fliver_20210929.h5ad')
data2 = sc.read('/home/jovyan/wgeted_files/gastrulation.h5ad')
liver_rest = sc.read('/home/jovyan/mount_farm/nfs/team298/ar32/YS/All_fetal_liver_data/A5_fetal_liver_alladata_updated_annots_210721.h5ad')
liver_adult = sc.read('/home/jovyan/mount_farm/nfs/team298/ar32/YS/adult_liver/data_set1_sharma_not_raw/Liver_raw_20220303.h5ad')

In [ ]:
data1.obs['celltypes_for_dotplot'] = data1.obs['broad_cell_labels_organ']
data2.obs['celltypes_for_dotplot'] = data2.obs['corr_concat']

adata_list = [data1,data2]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

cells_to_keep = ['Endoderm_ys','Fibroblast_ys','Smooth_Muscle_ys','Mesothelium_ys','Hepatocyte_fliv','Fibroblast_fliv','YS Endoderm_Yolk Sac_gastrulation']
adata_keep = adata[adata.obs['celltypes_for_dotplot'].isin(cells_to_keep)]
adata = adata_3a[:]

In [ ]:
liver_rest = liver_rest[~liver_rest.obs['orig.ident'].isin(['F61', 'F35', 'F32'])] # remove any cells under 'F61', 'F35', 'F32' as in combined YS_EL object
liver_rest = liver_rest[liver_rest.obs['cell.labels'].isin(['Hepatocyte'])] #,'Fibroblast'
liver_rest.obs['celltypes_for_dotplot'] = liver_rest.obs['cell.labels'].astype(str) + '_liver_not_matched'

adata_list = [adata,liver_rest]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

In [ ]:
liver_adult = liver_adult[liver_adult.obs['NormalvsTumor'].isin(['Normal'])]
liver_adult

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value = 10)

sc.pp.normalize_total(liver_adult)
sc.pp.log1p(liver_adult)
sc.pp.scale(liver_adult, max_value = 10)

In [ ]:
liver_adult = liver_adult[liver_adult.obs['HCCF1F2_anno'].isin(['Hepatocytes'])]
liver_adult.obs['celltypes_for_dotplot'] = liver_adult.obs['HCCF1F2_anno'].astype(str) + '_adult'

adata_list = [adata,liver_adult]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

In [ ]:
gene_list_plot = [
'SERPINA1',
'APOA2',
'APOC3',
'SPINK1',
'AHSG',
'AFP',
'APOA1',
'ALB',
'TTR',
'FGB',
'LUM',
'FRZB',
'COL3A1',
'SPARC',
'COL6A2',
'PTN',
'COL1A1',
'VIM',
'TPM1',
'CXCL12',
'CSRP2',
'CTHRC1',
'TPM2',
'CALD1',
'ACTA2',
'KRT19',
'S100A10',
'KRT8',
'TMEM98',
'PDPN',
'UPK3B'
]

adata.obs['celltypes_for_dotplot'] = adata.obs['celltypes_for_dotplot'].astype('category')
adata.obs['celltypes_for_dotplot'] = adata.obs['celltypes_for_dotplot'].cat.reorder_categories(['Endoderm_ys','YS Endoderm_Yolk Sac_gastrulation','Hepatocyte_fliv','Hepatocyte_liver_not_matched','Hepatocytes_adult','Fibroblast_ys','Fibroblast_fliv','Smooth_Muscle_ys','Mesothelium_ys'])

sc.pl.dotplot(adata, groupby= 'celltypes_for_dotplot', var_names=gene_list_plot, use_raw=False, swap_axes=False, figsize=(10,5), save = 'fig4a_with_adult_20220310.pdf')